In [2]:
# import packages
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
# loading the data
df = pd.read_csv('esb_day_level.csv')
df.head()

,vehicle_day_id,Region,Fleet,Vehicle ID,Date,dist,runtime,idling_time,idling_mins,driving_time,SOC,energy,temp,tdiff,over65,ptemp,speed,Efficiency,Heater,elec_heat,region_NC,region_SC,region_RM
0,4,NC,NC1 Lion,EV053,9/24/2021,6.265156,0.566389,0.001667,0.100000,0.564722,16.008713,33.618298,99.248000,34.248,1,34.248,11.089115,5.365916,electric,1,1,0,0
1,5,NC,NC1 Lion,EV053,11/26/2021,19.925346,1.511944,0.101111,6.066667,1.410833,71.075314,149.258159,45.842000,-19.158,0,0,9.504779,7.490869,electric,1,1,0,0
2,6,NC,NC1 Lion,EV053,5/26/2022,8.562315,0.614722,0.025833,1.550000,0.588889,19.634705,41.232881,74.078047,9.07804692,1,9.07804692,8.664240,4.815623,electric,1,1,0,0
3,8,NC,NC1 Lion,EV055,12/3/2021,7.036539,0.915833,0.071667,4.300000,0.844167,16.979738,35.657451,50.192000,-14.808,0,0,7.929321,5.067470,electric,1,1,0,0
4,9,NC,NC1 Lion,EV055,3/23/2022,13.029160,0.647778,0.098611,5.916667,0.549167,27.995041,58.789586,66.002000,1.002,1,1.002,23.714858,4.512155,electric,1,1,0,0


In [4]:
len(df)

2980

In [5]:
df['Region'].unique()

array(['NC', 'SC', 'RM', 'GL'], dtype=object)

# Data Cleaning

In [6]:
#drop unnecessary columns
cleaned_df = df.drop(['vehicle_day_id', 'Region', 'Fleet', 'Date', 'Heater'], axis=1)

In [7]:
#drop aberrant rows: e.g. SOC > 100%

In [8]:
#cleaned_df[cleaned_df['SOC'] > 100]

In [69]:
rs = np.random.RandomState(0)
df = pd.DataFrame(rs.rand(10, 10))
corr = cleaned_df.corr()
corr.style.background_gradient(cmap='coolwarm')

,dist,runtime,idling_time,idling_mins,driving_time,SOC,energy,temp,over65,speed,Efficiency,elec_heat,region_NC,region_SC,region_RM
dist,1.000000,0.769292,0.357321,0.357321,0.806210,0.875948,0.759459,-0.306842,-0.169178,0.178791,-0.436957,-0.500536,-0.535726,-0.031345,0.025672
runtime,0.769292,1.000000,0.695074,0.695074,0.930522,0.706952,0.678989,-0.238855,-0.208311,-0.152981,-0.265211,-0.430099,-0.408025,-0.101519,0.114835
idling_time,0.357321,0.695074,1.000000,1.000000,0.383481,0.268676,0.098696,-0.325826,-0.211667,-0.439911,-0.274831,-0.499264,-0.217084,-0.369106,0.259446
idling_mins,0.357321,0.695074,1.000000,1.000000,0.383481,0.268676,0.098696,-0.325826,-0.211667,-0.439911,-0.274831,-0.499264,-0.217084,-0.369106,0.259446
driving_time,0.806210,0.930522,0.383481,0.383481,1.000000,0.750764,0.792294,-0.128699,-0.159771,0.027576,-0.199384,-0.298175,-0.413565,0.057615,0.015352
SOC,0.875948,0.706952,0.268676,0.268676,0.750764,1.000000,0.873556,-0.154455,-0.056735,0.216197,-0.095016,-0.153590,-0.438509,0.201544,0.163491
energy,0.759459,0.678989,0.098696,0.098696,0.792294,0.873556,1.000000,0.060588,0.034152,0.362436,0.063318,0.131687,-0.436146,0.461955,0.032420
temp,-0.306842,-0.238855,-0.325826,-0.325826,-0.128699,-0.154455,0.060588,1.000000,0.739298,0.247724,0.283623,0.555561,0.145959,0.451464,-0.150491
over65,-0.169178,-0.208311,-0.211667,-0.211667,-0.159771,-0.056735,0.034152,0.739298,1.000000,0.208632,0.076972,0.357766,-0.039070,0.383398,-0.075701
speed,0.178791,-0.152981,-0.439911,-0.439911,0.027576,0.216197,0.362436,0.247724,0.208632,1.000000,-0.031797,0.363490,-0.251718,0.560123,-0.074755


In [70]:
##correlation of every feature with indepdent variable of idling_mins
##runtime is the strongest correlated feature with idling_mins
#cleaned_df.corr()['idling_mins']

In [71]:
##correlation of every feature with dependent variable of energy
#cleaned_df.corr()['energy']

In [72]:
#types of buses in the dataset
cleaned_df['Vehicle ID'].unique()

array(['EV053', 'EV055', 'EV056', 'EV057', 'EV058', 'EV059', 'EV060',
       'EV054', 'EV061', 'EV062', 'EV063', 'EV064', 'INTL01', 'INTL02',
       'INTL03', 'INTL04', 'INTL05', 'INTL06', 'INTL07', 'INTL08',
       'INTL09', 'INTL10', 'BLBR01', 'BLBR02', 'BLBR03', 'EV146', 'EV170',
       'EV171', 'EV172', 'EV173'], dtype=object)

In [ ]:
cleaned_df['Vehicle ID'].unique()

In [73]:
cleaned_df['dist'].describe()

count    2980.000000
mean       60.534678
std        40.106137
min         5.003363
25%        27.402926
50%        56.000000
75%        84.000000
max       205.000000
Name: dist, dtype: float64

In [74]:
#all variables that are not correlated with idling_time = dist, over65, elec_heat, region_NC, region_SC, region_RM

# Base Regression

In [103]:
#don't include region_RM since that is serving as the 
model = smf.ols(formula='energy ~  dist + idling_mins + over65 + elec_heat + region_NC + region_SC + idling_percentage', data=cleaned_df).fit()
# # model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 energy   R-squared:                       0.823
Model:                            OLS   Adj. R-squared:                  0.822
Method:                 Least Squares   F-statistic:                     1707.
Date:                Wed, 07 Feb 2024   Prob (F-statistic):               0.00
Time:                        15:37:21   Log-Likelihood:                -11720.
No. Observations:                2581   AIC:                         2.346e+04
Df Residuals:                    2573   BIC:                         2.350e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           -48.4879      2.22

## Interpretation: this regression has a strong R^2 of 0.82. All of the variables are significant. 

## for each unit increase in idling minutes (1 minute idled), the energy usage of the battery inceases by .15 kWh

# Using New Features in Regression

In [107]:
cleaned_df['SOC'].describe()

count    2526.000000
mean       50.314264
std        27.523115
min         0.950443
25%        29.038793
50%        48.352475
75%        68.400000
max       265.600000
Name: SOC, dtype: float64

In [116]:
cleaned_df['Efficiency'].describe()

count    2581.000000
mean        1.942760
std         0.895594
min         0.306302
25%         1.496239
50%         1.861605
75%         2.255526
max        16.328469
Name: Efficiency, dtype: float64

In [119]:
##engineering new features
#create a new column called idling percentage
cleaned_df['idling_percentage'] = (cleaned_df['idling_time'] / cleaned_df['runtime'])
#add a categorical column where 
cleaned_df.loc[cleaned_df['idling_percentage'] > 0.5, 'idling_more_than_half'] = 1
cleaned_df.loc[cleaned_df['idling_percentage'] <= 0.5, 'idling_more_than_half'] = 0
#create a new column for energy per mile
cleaned_df.loc[cleaned_df['Efficiency'] > 1.94, 'efficency_more_than_mean'] = 1
cleaned_df.loc[cleaned_df['Efficiency'] <= 1.94, 'efficency_more_than_mean'] = 0
#create a new column for SOC more than 1/2
cleaned_df.loc[cleaned_df['SOC'] > 50, 'soc_more_than_half'] = 1
cleaned_df.loc[cleaned_df['SOC'] <= 50, 'soc_more_than_half'] = 0
cleaned_df.head()

,Vehicle ID,dist,runtime,idling_time,idling_mins,driving_time,SOC,energy,temp,tdiff,over65,ptemp,speed,Efficiency,elec_heat,region_NC,region_SC,region_RM,idling_percentage,idling_more_than_half,soc_more_than_half,efficency_more_than_mean
0,EV053,6.265156,0.566389,0.001667,0.100000,0.564722,16.008713,33.618298,99.248000,34.248,1,34.248,11.089115,5.365916,1,1,0,0,0.002943,0.0,0.0,1.0
1,EV053,19.925346,1.511944,0.101111,6.066667,1.410833,71.075314,149.258159,45.842000,-19.158,0,0,9.504779,7.490869,1,1,0,0,0.066875,0.0,1.0,1.0
2,EV053,8.562315,0.614722,0.025833,1.550000,0.588889,19.634705,41.232881,74.078047,9.07804692,1,9.07804692,8.664240,4.815623,1,1,0,0,0.042024,0.0,0.0,1.0
3,EV055,7.036539,0.915833,0.071667,4.300000,0.844167,16.979738,35.657451,50.192000,-14.808,0,0,7.929321,5.067470,1,1,0,0,0.078253,0.0,0.0,1.0
4,EV055,13.029160,0.647778,0.098611,5.916667,0.549167,27.995041,58.789586,66.002000,1.002,1,1.002,23.714858,4.512155,1,1,0,0,0.152230,0.0,0.0,1.0


In [120]:
cleaned_df['idling_more_than_half'].value_counts()

0.0    2750
1.0     230
Name: idling_more_than_half, dtype: int64

In [121]:
cleaned_df['soc_more_than_half'].value_counts()

0.0    1307
1.0    1219
Name: soc_more_than_half, dtype: int64

In [124]:
#don't include region_RM since that is serving as the 
model = smf.ols(formula='energy ~  dist + idling_mins + over65 + elec_heat + region_NC + region_SC + idling_percentage + soc_more_than_half', data=cleaned_df).fit()
# # model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 energy   R-squared:                       0.844
Model:                            OLS   Adj. R-squared:                  0.843
Method:                 Least Squares   F-statistic:                     1697.
Date:                Wed, 07 Feb 2024   Prob (F-statistic):               0.00
Time:                        16:00:49   Log-Likelihood:                -11306.
No. Observations:                2524   AIC:                         2.263e+04
Df Residuals:                    2515   BIC:                         2.268e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept            -36.9137      2

# Plotting Variables Against Energy